In [1]:
require 'statsample-glm'
require 'open-uri'

content = open('http://www.ats.ucla.edu/stat/data/binary.csv')
File.write('binary.csv', content.read)

df = Daru::DataFrame.from_csv "binary.csv"
df.vectors = Daru::Index.new(['admit', 'gpa', 'gre', 'rank'])
df.to_category 'rank'
df.first 5

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"https://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"https://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    w

#<Daru::DataFrame(5x4)>
       admit   gpa   gre  rank
     0     0   380  3.61     3
     1     1   660  3.67     3
     2     1   800     4     1
     3     1   640  3.19     4
     4     0   520  2.93     4

In [2]:
formula = 'admit ~ gpa + gre + rank'
glm = Statsample::GLM::Regression.new formula, df, :logistic
glm.model.coefficients :hash

{:rank_1=>1.34020391646789, :rank_4=>-0.21125976045018074, :rank_2=>0.6647609885043279, :gpa=>0.0022644257861791682, :gre=>0.8040375492802225, :constant=>-5.330182989798933}

In [3]:
new_data = Daru::DataFrame.new({
  'gre' => [df['gre'].mean]*4,
  'gpa' => [df['gpa'].mean]*4,
  'rank' => [1, 2, 3, 4]
  })

#<Daru::DataFrame(4x3)>
                   gpa        gre       rank
          0      587.7 3.38990000          1
          1      587.7 3.38990000          2
          2      587.7 3.38990000          3
          3      587.7 3.38990000          4

In [4]:
glm.predict new_data

#<Daru::Vector(4)>
                   0  0.5166016075057968
                   1 0.35228458910369087
                   2 0.21861204944825496
                   3 0.18466844365413795

### Does a better formula exist?

In [5]:
formula = 'admit ~ gpa*gre + rank*gpa + rank*gre'
glm = Statsample::GLM::Regression.new formula, df, :logistic
glm.model.coefficients :hash

{:rank_1=>-0.7173647515003666, :rank_4=>-1.761778678428164, :rank_2=>0.26223307721996897, :gpa=>0.021519839510950503, :"rank_1:gpa"=>-0.0013610884959176224, :"rank_2:gpa"=>-0.0024319643973897075, :"rank_4:gpa"=>-0.001128103464968383, :gre=>3.4789198771103758, :"rank_1:gre"=>0.8249952288402159, :"rank_2:gre"=>0.5204610321760272, :"rank_4:gre"=>0.6223176111144127, :"gpa:gre"=>-0.005226643835763783, :constant=>-15.172413286712715}

In [6]:
glm.predict new_data

#<Daru::Vector(4)>
                   0  0.5336019579558202
                   1  0.3664479457989552
                   2 0.24144673351900073
                   3 0.18847730467585333